#deberta-v3-base-abas-v1.1
https://medium.com/nlplanet/quick-intro-to-aspect-based-sentiment-analysis-c8888a09eda7

- the dataset is unlabeled, this study uses a pre-trained ABSA model yangheng/deberta-v3-base-absa-v1.1 to generate pseudo ground truth sentiment labels.
- this model trained on SemEval datasets, and can detect aspect terms and classify the associated sentiment into pos/neu/neg
- DeBERTa assign sentiment polarity and cofidence score (for fine tuning and evaluation)


####Using a public pre-trained model
First, we install the transformers library along with the SentencePiece tokenizer (which is needed by some models of the library, such as DeBERTa

In [ ]:
!pip install transformers[sentencepiece]

Load Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch
import pandas as pd
from tqdm import tqdm

# Load the tokenizer and ABSA model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [ ]:
# Load the CSV (update path if needed)
df = pd.read_csv("ASPECT_preprocessed.csv")

# Keep only needed columns and remove duplicates
df = df[['Id', 'cleaned_text', 'aspect']].drop_duplicates()


In [ ]:
df.shape

(5329, 3)

In [ ]:
# Set model to eval mode
absa_model.eval()

# Store results here
results = []

# Loop through each row
for _, row in tqdm(df.iterrows(), total=len(df)):
    sentence = row['cleaned_text']
    aspect = row['aspect']

    # ABSA-specific input format
    encoded_input = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")

    with torch.no_grad():
        output = absa_model(**encoded_input)
        probs = F.softmax(output.logits, dim=1).detach().numpy()[0]

    # Map probabilities to sentiment
    sentiment_labels = ['negative', 'neutral', 'positive']
    sentiment = sentiment_labels[probs.argmax()]
    confidence = probs.max()

    results.append({
        "Id": row['Id'],
        "cleaned_text": sentence,
        "aspect": aspect,
        "sentiment": sentiment,
        "confidence": confidence
    })


100%|██████████| 5329/5329 [51:37<00:00,  1.72it/s]


In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df.head()

,Id,cleaned_text,aspect,sentiment,confidence
0,1,malaysia is a muslim country they support pale...,Islamic Identity,negative,0.794209
1,1,malaysia is a muslim country they support pale...,Race,negative,0.924682
2,1,malaysia is a muslim country they support pale...,National Identity,negative,0.875617
3,2,in kerala a hindu politician visiting a temple...,Islamic Identity,negative,0.724756
4,2,in kerala a hindu politician visiting a temple...,Secularism,negative,0.793947


In [ ]:
# Convert to DataFrame and save
results_df.to_csv("DeBERTa.csv", index=False)

In [ ]:
results_df.shape

(5329, 5)

####Merge file


In [ ]:
import pandas as pd

# Load the datasets
aspect_df = pd.read_csv("ASPECT_preprocessed.csv")
deberta_df = pd.read_csv("DeBERTa.csv")

# Merge on 'Id', 'translated_text', and 'aspect'
merged_df = pd.merge(
    aspect_df[['Id', 'created_at', 'cleaned_text', 'type', 'aspect']],
    deberta_df[['Id', 'cleaned_text', 'aspect', 'sentiment', 'confidence']],
    # Remove 'type' from the 'on' list as it's not in both dataframes
    on=['Id', 'cleaned_text', 'aspect'],
    how='inner'
)

# Rename 'sentiment' to 'deberta_sentiment'
merged_df.rename(columns={'sentiment': 'deberta_sentiment'}, inplace=True)

# Reorder columns
merged_df = merged_df[['Id', 'created_at', 'cleaned_text', 'type', 'aspect', 'deberta_sentiment', 'confidence']]

In [ ]:
# Save the result to a new CSV file
merged_df.to_csv("ABSA-DeBERTa_annotate.csv", index=False)

In [ ]:
merged_df.shape

(5329, 7)